### Imports

In [1]:
import cv2
import numpy as np
from pathlib import Path
from dataclasses import dataclass, field
import random
import shutil
import concurrent.futures
import os
import shutil
import tkinter as tk
import ctypes
import platform
import tensorflow as tf
import gc
from tensorflow.keras.models import load_model


2026-01-22 18:13:48.633000: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-22 18:13:48.675686: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-22 18:13:49.797793: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
mkdir -p failed for path /mnt/DatenUbuntu/marlon_home/.cache/matplotlib: [Errno 13] Permiss

### Alte Patches, Bilder, Heatmaps und Results löschen

In [2]:
folders = ['test_picture', 'test_patches', 'heatmaps_output', 'final_results']

for folder in folders:
    if not os.path.exists(folder):
        print(f"Folder does not exist: {folder}")
        continue

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Delete file
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Delete folder
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

### Bildvorverarbeitung

In [ ]:

@dataclass
class MultiScaleConfig:
    patch_size: int = 256
    scale_divisors: list = field(default_factory=lambda: [1.5, 2.0, 3.0, 4.0, 6.0])
    overlap: float = 0.5 
    use_relative_min_size: bool = True
    min_relative_factor: float = 0.15
    absolute_pixel_floor: int = 128
    interpolation: int = cv2.INTER_AREA # Schnell & gut für Verkleinerung
    show_visualization: bool = False  # <--- HIER UMSCHALTEN

# --- Optimierte Kern-Logik ---

def process_single_image(img_file, cfg, out_patches_root, is_visual=True):
    """
    Verarbeitet ein Bild. Wenn is_visual=True, wird ein Vorschaubild erzeugt.
    """
    img = cv2.imread(str(img_file))
    if img is None: return [], None

    h_orig, w_orig = img.shape[:2]
    base_size = min(h_orig, w_orig)
    limit_size = max(int(base_size * cfg.min_relative_factor), cfg.absolute_pixel_floor) if cfg.use_relative_min_size else 256
    
    img_patch_dir = out_patches_root / img_file.stem
    img_patch_dir.mkdir(parents=True, exist_ok=True)
    
    vis_img = img.copy() if is_visual else None
    colors = [(255, 0, 0), (255, 255, 0), (0, 255, 0), (0, 255, 255), (0, 165, 255)]
    
    local_metadata = []
    patch_count = 0

    for scale_idx, divisor in enumerate(cfg.scale_divisors):
        win_size = max(int(base_size / divisor), limit_size)
        win_size = min(win_size, base_size)
        stride = max(1, int(win_size * (1 - cfg.overlap)))
        
        # Einmaliges Padding pro Skalierung spart massiv Zeit
        # Wir padden so viel, dass wir beim Slicing keine Fehler bekommen
        pad = win_size 
        img_padded = cv2.copyMakeBorder(img, 0, pad, 0, pad, cv2.BORDER_CONSTANT, value=[0,0,0])

        for y in range(0, h_orig - win_size + stride, stride):
            for x in range(0, w_orig - win_size + stride, stride):
                # Slice direkt aus dem gepaddeten Bild (blitzschnell)
                patch = img_padded[y : y + win_size, x : x + win_size]
                
                if patch.shape[0] != cfg.patch_size:
                    patch = cv2.resize(patch, (cfg.patch_size, cfg.patch_size), interpolation=cfg.interpolation)
                
                patch_name = f"S{scale_idx}_p{patch_count}.jpg"
                cv2.imwrite(str(img_patch_dir / patch_name), patch)
                
                local_metadata.append(f"{img_file.name};{img_file.stem}/{patch_name};{x};{y};{win_size};{w_orig};{h_orig}")
                
                if is_visual:
                    color = colors[scale_idx % len(colors)]
                    cv2.rectangle(vis_img, (x, y), (x + win_size, y + win_size), color, max(1, int(win_size/200)))
                
                patch_count += 1
                
    return local_metadata, vis_img

def main():
    # --- PFADE ---
    input_dir = Path("~/DatenUbuntu/Studium/1.Semester/KI-Projekt/modeltest/Testbilder_Datensatz/datensatz").expanduser()
    test_picture_dir = Path("test_picture") 
    output_root = Path("test_patches")    
    cfg = MultiScaleConfig()

    # Vorbereitung
    if test_picture_dir.exists(): shutil.rmtree(test_picture_dir)
    if output_root.exists(): shutil.rmtree(output_root)
    test_picture_dir.mkdir(parents=True, exist_ok=True)
    out_patches_root = output_root / "patches"
    out_patches_root.mkdir(parents=True, exist_ok=True)

    img_files = list(input_dir.glob("*.[jJ][pP][gG]")) + list(input_dir.glob("*.[pP][nN][gG]"))
    if not img_files: return

    user_input = input(f"Bilder (Zahl oder 'all' [Gesamt: {len(img_files)}]): ")
    num = len(img_files) if user_input.lower() == 'all' else int(user_input)
    selected_files = random.sample(img_files, min(num, len(img_files)))

    all_patch_metadata = []

    # --- MODUS ENTSCHEIDUNG ---
    if cfg.show_visualization:
        print("📺 Visueller Modus (Sequenziell)... 'q' zum Abbrechen.")
        for f in selected_files:
            meta, vis = process_single_image(f, cfg, out_patches_root, is_visual=True)
            all_patch_metadata.extend(meta)
            shutil.copy2(f, test_picture_dir / f.name)
            
            # Anzeige
            h, w = vis.shape[:2]
            scale = 800 / max(h, w)
            cv2.imshow("Preview", cv2.resize(vis, (int(w*scale), int(h*scale))))
            print(f"✅ {f.name} - {len(meta)} Patches")
            if cv2.waitKey(1) & 0xFF == ord('q'): break
        cv2.destroyAllWindows()
    
    else:
        print(f"🚀 Turbo-Modus: Parallele Verarbeitung auf {os.cpu_count()} Kernen...")
        with concurrent.futures.ProcessPoolExecutor() as executor:
            futures = {executor.submit(process_single_image, f, cfg, out_patches_root, False): f for f in selected_files}
            for future in concurrent.futures.as_completed(futures):
                f = futures[future]
                meta, _ = future.result()
                all_patch_metadata.extend(meta)
                shutil.copy2(f, test_picture_dir / f.name)
                print(f"✅ {f.name} - {len(meta)} Patches")

    # Metadaten speichern
    with open(output_root / "metadata.txt", "w") as f:
        f.write("\n".join(all_patch_metadata))
    
    print(f"\nFertig! Patches in '{output_root}' gespeichert.")

if __name__ == "__main__":
    main()

🚀 Turbo-Modus: Parallele Verarbeitung auf 16 Kernen...
✅ IMG_2977_crop_1306x599.jpg - 177 Patches
✅ IMG_2905_crop_1863x1213.jpg - 154 Patches
✅ IMG_2822_crop_2399x992.jpg - 232 Patches
✅ IMG_2943_crop_1642x1913.jpg - 115 Patches
✅ IMG_3010_crop_2706x1406.jpg - 187 Patches
✅ IMG_2929_crop_3005x2006.jpg - 143 Patches
✅ IMG_2895_crop_2813x2127.jpg - 128 Patches
✅ IMG_3012_crop_1685x2427.jpg - 143 Patches
✅ IMG_2991_crop_3105x2084.jpg - 143 Patches
✅ IMG_2813_crop_2206x1012.jpg - 210 Patches
✅ IMG_2967_crop_2770x2556.jpg - 110 Patches
✅ IMG_2885_crop_3527x2092.jpg - 166 Patches
✅ IMG_2863_crop_3641x1642.jpg - 210 Patches
✅ IMG_2854_crop_2734x1677.jpg - 159 Patches
✅ IMG_2940_crop_2806x2427.jpg - 115 Patches
✅ IMG_2843_crop_4508x2083.jpg - 210 Patches
✅ IMG_2953_crop_1842x2106.jpg - 110 Patches
✅ IMG_2917_crop_1270x599.jpg - 167 Patches
✅ IMG_2839_crop_3848x2834.jpg - 128 Patches
✅ IMG_2870_crop_3877x3191.jpg - 122 Patches
✅ IMG_2957_crop_1863x1071.jpg - 166 Patches
✅ IMG_2959_crop_3898x229

### Modell

In [6]:

# --- GPU Setup ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' # Falls nötig aktivieren

def setup_gpu():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU Beschleunigung aktiv.")
    except: pass
setup_gpu()

class QRAllInOneConfig:
    model_path = 'final_model.keras'
    base_path = Path('test_patches')
    patch_folder = base_path / "patches"
    metadata_file = base_path / "metadata.txt"
    original_img_dir = Path('test_picture')
    
    # Ausgabe-Ordner
    output_dir_final = Path('final_results')
    output_dir_heat = Path('heatmaps_output')
    
    # Parameter
    min_vote_prob = 0.3    # Ab wann zählt ein Patch zur Heatmap?
    vote_threshold = 5     # Heatmap-Wert für "Box zeichnen"
    fixed_vis_max = 25.0   # Skalierungswert für die Heatmap-PNG (für den Viewer)
    
    start_batch_size = 32 

def run_pipeline():
    cfg = QRAllInOneConfig()
    cfg.output_dir_final.mkdir(exist_ok=True)
    cfg.output_dir_heat.mkdir(exist_ok=True) # Heatmap Ordner erstellen
    
    if not cfg.metadata_file.exists():
        print("❌ Metadaten-Datei nicht gefunden!")
        return

    print("⏳ Lade Modell...")
    model = load_model(cfg.model_path, compile=False)
    
    # Metadaten einlesen
    with open(cfg.metadata_file, "r") as f:
        lines = [l.strip().split(";") for l in f.readlines()]

    images_dict = {}
    for img_name, rel, px, py, ps, w_orig, h_orig in lines:
        if img_name not in images_dict:
            images_dict[img_name] = {"w": int(w_orig), "h": int(h_orig), "patches": []}
        images_dict[img_name]["patches"].append({"path": rel, "x": int(px), "y": int(py), "s": int(ps)})

    print(f"--- Starte Kombi-Pipeline (Heatmaps + Boxen) ---")

    for img_name, info in images_dict.items():
        orig_img = cv2.imread(str(cfg.original_img_dir / img_name))
        if orig_img is None: continue
        
        h_orig, w_orig = info["h"], info["w"]
        patch_list = info["patches"]
        
        # --- Batch-Verarbeitung ---
        all_patch_imgs = []
        for p in patch_list:
            p_img = cv2.imread(str(cfg.patch_folder / p["path"]), cv2.IMREAD_GRAYSCALE)
            if p_img is None: continue
            p_img = cv2.resize(p_img, (256, 256))
            all_patch_imgs.append(p_img.astype(np.float32))

        if not all_patch_imgs: continue
        input_batch = np.expand_dims(np.array(all_patch_imgs), axis=-1)

        # Vorhersage
        preds = None
        current_bs = cfg.start_batch_size
        while current_bs >= 1:
            try:
                preds = model.predict(input_batch, batch_size=current_bs, verbose=0)
                break 
            except tf.errors.ResourceExhaustedError:
                current_bs //= 2
                gc.collect()
        
        if preds is None: continue

        # --- Heatmap Berechnung ---
        heatmap_sum = np.zeros((h_orig, w_orig), dtype=np.float32)
        for i, prob_vec in enumerate(preds):
            prob = float(prob_vec[0]) if len(prob_vec) == 1 else float(prob_vec[1])
            if prob > cfg.min_vote_prob:
                p = patch_list[i]
                heatmap_sum[p["y"]:p["y"]+p["s"], p["x"]:p["x"]+p["s"]] += prob

        # ---------------------------------------------------------
        # SCHRITT 1: Heatmap speichern (für den Viewer)
        # ---------------------------------------------------------
        # Normierung genau wie im Viewer erwartet (auf Basis von 25.0)
        heatmap_norm = (heatmap_sum / cfg.fixed_vis_max) * 255.0
        heatmap_8bit = np.clip(heatmap_norm, 0, 255).astype(np.uint8)
        
        heat_out_path = cfg.output_dir_heat / f"{Path(img_name).stem}_heatmap.png"
        cv2.imwrite(str(heat_out_path), heatmap_8bit)

        # ---------------------------------------------------------
        # SCHRITT 2: Boxen zeichnen (für das Ergebnisbild)
        # ---------------------------------------------------------
        _, thresh = cv2.threshold(heatmap_sum, cfg.vote_threshold, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        result_img = orig_img.copy()
        found_count = 0
        
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 30 and h > 30: 
                cv2.rectangle(result_img, (x, y), (x + w, y + h), (0, 255, 0), 4)
                # cv2.putText(result_img, "QR", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                found_count += 1

        final_out_path = cfg.output_dir_final / f"final_{img_name}"
        cv2.imwrite(str(final_out_path), result_img)
        
        print(f"✅ {img_name}: {found_count} Objekte | Heatmap & Bild gespeichert.")

        del input_batch, all_patch_imgs, preds, heatmap_sum
        gc.collect()

    tf.keras.backend.clear_session()
    print(f"\nFertig! Viewer kann jetzt gestartet werden.")

if __name__ == "__main__":
    run_pipeline()

✅ GPU Beschleunigung aktiv.
⏳ Lade Modell...


I0000 00:00:1769101880.156897   82004 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4081 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


--- Starte Kombi-Pipeline (Heatmaps + Boxen) ---


2026-01-22 18:11:21.592223: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91701


✅ IMG_2978_crop_1392x564.jpg: 1 Objekte | Heatmap & Bild gespeichert.


KeyboardInterrupt: 

### Visualisierung

In [10]:
# --- DPI FIX FÜR WINDOWS ---
if platform.system() == "Windows":
    try:
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        ctypes.windll.user32.SetProcessDPIAware()

# --- 1. DESIGN & STANDARDS ---
TARGET_HEIGHT = 700  # Basis-Höhe für die Einzelbilder
HEADER_HEIGHT = 60
FOOTER_HEIGHT = 130
MARGIN = 20
FIXED_MAX_SCORE = 25.0 

C_BG = (24, 24, 27)
C_HEADER = (39, 39, 42)
C_ACCENT = (14, 165, 233)
C_TEXT = (228, 228, 231)
C_SUCCESS = (34, 197, 94)
C_FAIL = (239, 68, 68)

path_original = Path('test_picture')
path_heatmap = Path('heatmaps_output')
path_decision = Path('final_results')

def get_screen_size():
    root = tk.Tk()
    sw = root.winfo_screenwidth()
    sh = root.winfo_screenheight()
    root.destroy()
    return sw, sh

def resize_with_aspect(img, target_h):
    h, w = img.shape[:2]
    scale = target_h / h
    return cv2.resize(img, (int(w * scale), target_h), interpolation=cv2.INTER_AREA)

def show_interactive_evaluation():
    img_files = list(path_original.glob("*.[jJ][pP][gG]")) + list(path_original.glob("*.[pP][nN][gG]"))
    if not img_files:
        print("❌ Keine Bilder gefunden!")
        return
    
    random.shuffle(img_files)

    # Bildschirmmaße abrufen und Puffer lassen
    sw, sh = get_screen_size()
    print(f"Detected Screen Resolution: {sw}x{sh}")
    
    # Maximale Fenstergröße (90% der Breite, 80% der Höhe für Taskleiste/Fensterrahmen)
    max_w = int(sw * 0.90)
    max_h = int(sh * 0.85)

    stats = {"richtig": 0, "falsch": 0, "gesamt": 0}

    for idx, img_file in enumerate(img_files):
        # Daten laden
        img_orig = cv2.imread(str(img_file))
        heat_path = path_heatmap / f"{img_file.stem}_heatmap.png"
        img_heat_gray = cv2.imread(str(heat_path), cv2.IMREAD_GRAYSCALE) if heat_path.exists() else None
        img_final = cv2.imread(str(path_decision / f"final_{img_file.name}"))

        if img_orig is None or img_final is None: continue

        # Score Logik
        max_pixel_val = np.max(img_heat_gray) if img_heat_gray is not None else 0
        real_score = (max_pixel_val / 255.0) * FIXED_MAX_SCORE
        qr_found = real_score >= 5.0 

        # Einzelbilder skalieren
        res_orig = resize_with_aspect(img_orig, TARGET_HEIGHT)
        res_final = resize_with_aspect(img_final, TARGET_HEIGHT)
        if img_heat_gray is not None:
            res_heat_color = cv2.applyColorMap(resize_with_aspect(img_heat_gray, TARGET_HEIGHT), cv2.COLORMAP_JET)
        else:
            res_heat_color = np.zeros_like(res_orig)

        # Canvas berechnen
        content_w = res_orig.shape[1] + res_heat_color.shape[1] + res_final.shape[1] + (2 * MARGIN)
        content_h = HEADER_HEIGHT + TARGET_HEIGHT + FOOTER_HEIGHT
        
        canvas = np.full((content_h, content_w, 3), C_BG, dtype=np.uint8)

        # UI Elemente zeichnen (Header, Bilder, Footer)
        canvas[0:HEADER_HEIGHT, :] = C_HEADER
        curr_x = 0
        for title, w in [("ORIGINAL", res_orig.shape[1]), ("VOTE DENSITY", res_heat_color.shape[1]), ("FINAL", res_final.shape[1])]:
            cv2.putText(canvas, title, (curr_x + 10, 40), cv2.FONT_HERSHEY_DUPLEX, 0.6, C_ACCENT, 1, cv2.LINE_AA)
            curr_x += w + MARGIN

        canvas[HEADER_HEIGHT:HEADER_HEIGHT+TARGET_HEIGHT, 0:res_orig.shape[1]] = res_orig
        x_off = res_orig.shape[1] + MARGIN
        canvas[HEADER_HEIGHT:HEADER_HEIGHT+TARGET_HEIGHT, x_off:x_off+res_heat_color.shape[1]] = res_heat_color
        x_off += res_heat_color.shape[1] + MARGIN
        canvas[HEADER_HEIGHT:HEADER_HEIGHT+TARGET_HEIGHT, x_off:x_off+res_final.shape[1]] = res_final

        # Footer Texte
        f_y = HEADER_HEIGHT + TARGET_HEIGHT
        cv2.line(canvas, (0, f_y), (content_w, f_y), (63, 63, 70), 2)
        cv2.putText(canvas, f"FILE: {img_file.name} | {idx+1}/{len(img_files)}", (20, f_y + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (161, 161, 170), 1, cv2.LINE_AA)
        
        status_color = C_SUCCESS if qr_found else C_FAIL
        cv2.putText(canvas, "QR DETECTED" if qr_found else "NO DETECTION", (content_w - 320, f_y + 50), cv2.FONT_HERSHEY_DUPLEX, 0.7, status_color, 2, cv2.LINE_AA)
        cv2.putText(canvas, f"SCORE: {real_score:.2f}", (content_w - 320, f_y + 85), cv2.FONT_HERSHEY_SIMPLEX, 0.6, C_TEXT, 1, cv2.LINE_AA)
        cv2.putText(canvas, "[J] CORRECT   [N] WRONG   [Q] EXIT", (int(content_w/2) - 150, f_y + 110), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (113, 113, 122), 1, cv2.LINE_AA)

        # --- SKALIERUNG AUF BILDSCHIRMGRÖSSE ---
        final_canvas = canvas
        if content_w > max_w or content_h > max_h:
            scale = min(max_w / content_w, max_h / content_h)
            final_canvas = cv2.resize(canvas, (int(content_w * scale), int(content_h * scale)), interpolation=cv2.INTER_AREA)

        # --- FENSTER ANZEIGEN & ZENTRIEREN ---
        win_name = "AI Evaluation Dashboard"
        actual_w, actual_h = final_canvas.shape[1], final_canvas.shape[0]
        
        cv2.namedWindow(win_name, cv2.WINDOW_NORMAL) # WINDOW_NORMAL erlaubt Skalierung
        cv2.resizeWindow(win_name, actual_w, actual_h)
        cv2.moveWindow(win_name, (sw - actual_w) // 2, (sh - actual_h) // 2)
        
        cv2.imshow(win_name, final_canvas)
        
        key = cv2.waitKey(0) & 0xFF
        if key == ord('q'): break
        elif key == ord('j'): stats["richtig"] += 1; stats["gesamt"] += 1
        elif key == ord('n'): stats["falsch"] += 1; stats["gesamt"] += 1

    cv2.destroyAllWindows()
    if stats["gesamt"] > 0:
        print(f"\nFinal Accuracy: {(stats['richtig']/stats['gesamt']*100):.2f}%")

if __name__ == "__main__":
    show_interactive_evaluation()

Detected Screen Resolution: 3840x1200

Final Accuracy: 89.68%
